In [1]:
###############################################################
# 5-vertex connected graphs → images + automorphisms + A-matrix
# Everything in ONE PDF (patterns first, matrix next)
###############################################################

from sage.graphs.graph_generators import graphs
from itertools import permutations
from PIL import Image
import tempfile, os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from sage.graphs.graph import Graph

###############################################################
# Collect graphs (connected) and metadata
###############################################################

G5 = [G for G in graphs(5) if G.is_connected()]
m = len(G5)

labels = [f"G{i+1:03d}" for i in range(m)]
aut = [int(G.automorphism_group().cardinality()) for G in G5]

# Sort by automorphism count
order = sorted(range(m), key=lambda i: aut[i])
G5     = [G5[i] for i in order]
labels = [labels[i] for i in order]
aut    = [aut[i] for i in order]

print("Total connected graphs on 5 vertices:", m)

###############################################################
# Save PNG for each graph
###############################################################

img_files = []
for i, G in enumerate(G5, start=1):
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tname = tmp.name
    tmp.close()

    G.plot(figsize=3).save(tname)

    img = Image.open(tname).convert("RGBA")
    img.thumbnail((250, 250), Image.LANCZOS)

    out = f"p5_{i:03d}.png"
    img.save(out)
    img_files.append(out)

    os.remove(tname)

###############################################################
# Induced copy count (injective homomorphisms)
###############################################################

def embedding_count(P, H):
    k = P.order()
    P_vertices = list(P.vertices())
    H_vertices = list(H.vertices())
    P_edges = {(min(u,v), max(u,v)) for u,v,_ in P.edges()}

    count = 0
    for S in permutations(H_vertices, k):
        mapping = dict(zip(P_vertices, S))
        ok = True
        for u, v in P_edges:
            if not H.has_edge(mapping[u], mapping[v]):
                ok = False
                break
        if ok:
            count += 1
    return count

# Build A matrix
A = [[0]*m for _ in range(m)]
for i in range(m):
    for j in range(m):
        A[i][j] = embedding_count(G5[i], G5[j]) // aut[i]

###############################################################
# Build PDF (gallery + matrix)
###############################################################

pdf = "p5_full_corrected.pdf"
c = canvas.Canvas(pdf, pagesize=letter)
pw, ph = letter

margin = 0.5*inch
gap = 0.3*inch
cell_w = (pw - 2*margin - gap)/2
cell_h = (ph - 2*margin - gap)/2
header_h = 0.35*inch

# -------- Graph gallery --------
for idx in range(m):
    pos = idx % 4
    row = pos // 2
    col = pos % 2

    if pos == 0 and idx != 0:
        c.showPage()

    tx = margin + col*(cell_w + gap)
    top = ph - margin - row*(cell_h + gap)
    ty = top - cell_h

    c.setFont("Helvetica-Bold", 9)
    c.drawString(tx+5, top-12, f"{labels[idx]}   Aut: {aut[idx]}")

    box_w = cell_w - 0.2*inch
    box_h = cell_h - header_h - 0.2*inch

    img = Image.open(img_files[idx])
    w, h = img.size
    scale = min(box_w/w, box_h/h)
    dw, dh = w*scale, h*scale

    dx = tx + (cell_w - dw)/2
    dy = ty + (cell_h - header_h - dh)/2

    c.drawImage(img_files[idx], dx, dy, width=dw, height=dh)

c.showPage()

# -------- Matrix pages --------
c.setFont("Courier", 6)

x0 = 30
y0 = ph - 40
c.drawString(x0, y0, "A[i][j] = number of induced copies of Gi in Gj")
y = y0 - 20

row_h = 10
header = "     " + " ".join(f"{lab:>5}" for lab in labels)
c.drawString(x0, y, header)
y -= row_h

for i in range(m):
    row = f"{labels[i]:<5}" + " ".join(f"{A[i][j]:5d}" for j in range(m))
    c.drawString(x0, y, row)
    y -= row_h

    if y < 40:
        c.showPage()
        y = ph - 60
        c.setFont("Courier", 6)

c.showPage()
c.save()

print("DONE. Output PDF:", pdf)


Total connected graphs on 5 vertices: 21
DONE. Output PDF: p5_full_corrected.pdf


In [5]:
from sage.graphs.graph_generators import graphs

# Connected graphs on 5 vertices, ordered by automorphisms
G5 = [G for G in graphs(5) if G.is_connected()]
aut = [int(G.automorphism_group().cardinality()) for G in G5]


# Fixed edge order
edge_order = [
    (0,1), (0,2), (0,3), (0,4),
    (1,2), (1,3), (1,4),
    (2,3), (2,4),
    (3,4)
]

def edge_bitstring(G):
    return ''.join('1' if G.has_edge(u,v) else '0' for u,v in edge_order)

# Print ONLY matrix entries
for i, G in enumerate(G5, start=1):
    print(f"G{i} & {edge_bitstring(G)} &  &  &  &  \\\\")


G1 & 0001001011 &  &  &  &  \\
G2 & 0011001010 &  &  &  &  \\
G3 & 0011001011 &  &  &  &  \\
G4 & 0011011010 &  &  &  &  \\
G5 & 0011010011 &  &  &  &  \\
G6 & 0011011011 &  &  &  &  \\
G7 & 0011011110 &  &  &  &  \\
G8 & 0011011111 &  &  &  &  \\
G9 & 0101011000 &  &  &  &  \\
G10 & 0101011010 &  &  &  &  \\
G11 & 0101011011 &  &  &  &  \\
G12 & 0110011010 &  &  &  &  \\
G13 & 0111011010 &  &  &  &  \\
G14 & 0111011011 &  &  &  &  \\
G15 & 0111001110 &  &  &  &  \\
G16 & 0111001111 &  &  &  &  \\
G17 & 0111011111 &  &  &  &  \\
G18 & 0111111010 &  &  &  &  \\
G19 & 0111111011 &  &  &  &  \\
G20 & 0111111111 &  &  &  &  \\
G21 & 1111111111 &  &  &  &  \\
